# Train a simple deep NN on the MNIST dataset

In [35]:
#from _future_ import print_function
import keras
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.optimizers import RMSprop
from keras.utils import np_utils

batch_size = 128
num_classes = 10
epochs = 20

In [36]:
#Load and remake data
(x_train,y_train),(x_test,y_test) = mnist.load_data()
x_train = x_train.reshape(60000,784)
x_test = x_test.reshape(10000,784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
#归一化到0-1
x_train /= 255
x_test /= 255
print(x_train.shape[0],'train simples')
print(x_test.shape[0],'test simples')

60000 train simples
10000 test simples


In [37]:
# 将类别向量(从0到nb_classes的整数向量)映射为二值类别矩阵, 用于应用到以categorical_crossentropy为目标函数的模型中.
# y: 类别向量; nb_classes:总共类别数
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [38]:
# Dense层:即全连接层
# keras.layers.core.Dense(
#   output_dim, 
#   init='glorot_uniform', 
#   activation='linear', 
#   weights=None, 
#   W_regularizer=None,
#    b_regularizer=None, 
#    activity_regularizer=None, 
#    W_constraint=None, 
#   b_constraint=None, 
#   bias=True, 
#    input_dim=None)

#定义贯序模型
model = Sequential()
model.add(Dense(512,activation='linear',input_shape=(784,)))
# Dropout  需要断开的连接的比例---防止过拟合
model.add(Dropout(0.2))

model.add(Dense(512,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
# 打印出模型概况
print('model.summary:')
model.summary()

#在训练模型之前，通过compile来对学习过程进行配置
# 编译模型以供训练
# 包含评估模型在训练和测试时的性能的指标，典型用法是metrics=['accuracy']
# 如果要在多输出模型中为不同的输出指定不同的指标，可像该参数传递一个字典，例如metrics={'ouput_a': 'accuracy'}
model.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(),
             metrics=['accuracy'])
#训练模型
# Keras以Numpy数组作为输入数据和标签的数据类型
# fit(self, x, y, batch_size=32, 
# nb_epoch=10, verbose=1, callbacks=[], 
# validation_split=0.0,validation_data=None, 
# shuffle=True, class_weight=None, 
# sample_weight=None)
# nb_epoch：整数，训练的轮数，训练数据将会被遍历nb_epoch次。Keras中nb开头的变量均为"number of"的意思
# verbose：日志显示，0为不在标准输出流输出日志信息，1为输出进度条记录，2为每个epoch输出一行记录
# shuffle：布尔值，表示是否在训练过程中每个epoch前随机打乱输入样本的顺序。

# fit函数返回一个History的对象，其History.history属性记录了损失函数和其他指标的数值随epoch变化的情况，如果有验证集的话，也包含了验证集的这些指标变化情况
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Final Test loss:', score[0])
print('Final Test accuracy:', score[1])

model.summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 9s 148us/step

##  结果分析
dropout=0.2，dropout=0.2的结果：
Final Test loss: 0.111594470589
Final Test accuracy: 0.9835

dropout=0.2，dropout=0.3的结果：
Final Test loss: 0.108411227955
Final Test accuracy: 0.9832

dropout=0.1，dropout=0.1的结果：
Final Test loss: 0.12482077565
Final Test accuracy: 0.9833

dropout=0.2，dropout=0.2，epochs=30的结果：
Final Test loss: 0.137473855173
Final Test accuracy: 0.9835

dropout=0.2，dropout=0.2,激活函数采用relu的结果：
Final Test loss: 0.129076222247
Final Test accuracy: 0.9806